In [1]:
# Imports
import torch
from torch import nn

import wandb

from torch_geometric.datasets import AttributedGraphDataset
from torch_geometric.transforms import RandomNodeSplit

import torch.nn.modules.loss
import torch.optim.adam

from models import GCN_Model
from models import GAT_Model
from models import GCN_JK_Model
from models import GAT_JK_Model

from train import train_and_validate
from train import best_model
from train import test_on_testset
from train import train_and_test_model

In [ ]:
device = (
    "cuda" if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

In [ ]:
# Dataset
transform = RandomNodeSplit(split = "train_rest", num_val = 0.2, num_test = 0.2)
cora_dataset_raw = AttributedGraphDataset("C:/Users/lukas/Desktop/Advanced-topics-in-deep-learning/Assignments/Assignment 2/src/data", "Cora", transform = transform)
citeseer_dataset_raw = AttributedGraphDataset("C:/Users/lukas/Desktop/Advanced-topics-in-deep-learning/Assignments/Assignment 2/src/data", "CiteSeer", transform = transform)
cora_dataset = cora_dataset_raw[0].to(device)
citeseer_dataset = citeseer_dataset_raw[0].to(device)

In [4]:
epochs = 100
hidden_size = 24
num_classes = citeseer_dataset_raw.num_classes

Training_set = citeseer_dataset.subgraph(citeseer_dataset["train_mask"])
Validation_set = citeseer_dataset.subgraph(citeseer_dataset["val_mask"])
Test_set = citeseer_dataset.subgraph(citeseer_dataset["test_mask"])

#model = lambda l, d: GCN_Model(hidden_size, l, num_classes).to(device)
#model = lambda l, d: GAT_Model(d, hidden_size, l, num_classes).to(device)
#model = lambda l, d: GCN_JK_Model(hidden_size, l, "max", num_classes).to(device)
model = lambda l, d: GAT_JK_Model(d, hidden_size, l, "lstm", num_classes).to(device)

loss_fn = nn.CrossEntropyLoss()

best_performance = best_model(Training_set, Validation_set, model, loss_fn, 6, epochs)

In [ ]:
best_performance

In [ ]:
test_on_testset(Test_set, best_performance["model"], device)

In [7]:
Training_set = cora_dataset.subgraph(cora_dataset["train_mask"])
Validation_set = cora_dataset.subgraph(cora_dataset["val_mask"])
Test_set = cora_dataset.subgraph(cora_dataset["test_mask"])

num_classes = cora_dataset_raw.num_classes

hidden_size = 29

model = lambda l, d: GCN_Model(hidden_size, l, num_classes).to(device)
#model = lambda l, d: GAT_Model(d, hidden_size, l, num_classes).to(device)
#model = lambda l, d: GCN_JK_Model(hidden_size, l, "max", num_classes).to(device)
#model = lambda l, d: GAT_JK_Model(d, hidden_size, l, "lstm", num_classes).to(device)

MCAccuracy_mean, MCAccuracy_std = train_and_test_model(Training_set, Validation_set, Test_set, model, layers = 1, epochs = 10000, learning_rate = 0.0076, weight_decay = 0.00030, dropout = 0.3029, device = device)

In [ ]:
MCAccuracy_mean, MCAccuracy_std

In [6]:
def GCN_sweep(train_dataloader, val_dataloader, num_classes, config_=None):
  with wandb.init(config=config_):
    config_ = wandb.config

    model = lambda l, d: GCN_Model(config_.hidden_size, l, num_classes).to(device)

    loss_fn = nn.CrossEntropyLoss()

    MCAccuracies = []

    Use_Wandb = True

    for iteration in range(5):
      (_, MCAccuracy) = train_and_validate(train_dataloader, val_dataloader, model, loss_fn, 
                                          config_.layers, config_.epochs, config_.learning_rate, 
                                          config_.weight_decay, config_.dropout, iteration, Use_Wandb)
      
      MCAccuracies.append(MCAccuracy)

    MCAccuracies = torch.Tensor(MCAccuracies)

    Mean_MCAccuracy, STD_MCAccuracy = (torch.mean(MCAccuracies), torch.std(MCAccuracies))

    wandb.log({'validation mean accuracy': Mean_MCAccuracy})
    wandb.log({'validation std accuracy': STD_MCAccuracy})

In [7]:
def GAT_sweep(train_dataloader, val_dataloader, num_classes, config_=None):
  with wandb.init(config=config_):
    config_ = wandb.config

    model = lambda l, d: GAT_Model(d, config_.hidden_size, l, num_classes).to(device)

    loss_fn = nn.CrossEntropyLoss()

    MCAccuracies = []

    Use_Wandb = True

    for iteration in range(5):
      (_, MCAccuracy) = train_and_validate(train_dataloader, val_dataloader, model, loss_fn, 
                                          config_.layers, config_.epochs, config_.learning_rate, 
                                          config_.weight_decay, config_.dropout, iteration, Use_Wandb)
      
      MCAccuracies.append(MCAccuracy)

    MCAccuracies = torch.Tensor(MCAccuracies)

    Mean_MCAccuracy, STD_MCAccuracy = (torch.mean(MCAccuracies), torch.std(MCAccuracies))

    wandb.log({'validation mean accuracy': Mean_MCAccuracy})
    wandb.log({'validation std accuracy': STD_MCAccuracy})

In [8]:
# Sweep config
sweep_config = {
  'method': 'random',
  'parameters': {
    'hidden_size': {'distribution': 'int_uniform',
                                    'max': 32,
                                    'min': 16},
    'layers': {'distribution': 'int_uniform',
                               'max': 6,
                               'min': 1}, 
    'epochs': {'value': 100},
    'learning_rate': {'distribution': 'uniform',
                                      'max': 0.01,
                                      'min': 0},
    'weight_decay': {'distribution': 'uniform',
                                      'max': 0.001,
                                      'min': 0},
    'dropout': {'distribution': 'uniform',
                                'max': 1.0,
                                'min': 0.0},
  }
}

In [9]:
def Create_agent_GCN(train_dataloader, val_dataloader, num_classes):
  def GCN_sweep_agent(config_ = None):
    return GCN_sweep(train_dataloader, val_dataloader, num_classes, config_)

  sweep_id = wandb.sweep(sweep_config, project = "ATIDL Assignment 2")

  wandb.agent(sweep_id, function = GCN_sweep_agent, count = 20)

def Create_agent_GAT(train_dataloader, val_dataloader, num_classes):
  def GAT_sweep_agent(config_ = None):
    return GAT_sweep(train_dataloader, val_dataloader, num_classes, config_)

  sweep_id = wandb.sweep(sweep_config, project = "ATIDL Assignment 2")

  wandb.agent(sweep_id, function = GAT_sweep_agent, count = 20)

In [ ]:
Training_set = cora_dataset.subgraph(cora_dataset["train_mask"])
Validation_set = cora_dataset.subgraph(cora_dataset["val_mask"])

Create_agent_GAT(Training_set, Validation_set, cora_dataset_raw.num_classes)

In [ ]:
Training_set = cora_dataset.subgraph(cora_dataset["train_mask"])
Validation_set = cora_dataset.subgraph(cora_dataset["val_mask"])

Create_agent_GCN(Training_set, Validation_set, cora_dataset_raw.num_classes)